# Configurar área de trabajo y cargar datos
Usaremos las librerías de [Plotly](https://plotly.com/graphing-libraries/) para construir gráficos interactivos 📊✨.

In [ ]:
#borrar objetos previos
rm(list=ls())
#borrar figuras (si hay)
while(!dev.cur())
dev.off()
cat("\014")

# Para manipulación de bases de datos. Si no existe, la instalamos
if(!require(dplyr)){install.packages("dplyr")}
# Para importar/exportar bases de datos. Si no existe, la instalamos
if(!require(rio)){install.packages("rio")}
# Para exportar archivos en formato parquet
if(!require(arrow)){install.packages("arrow")}
if(!require(nanoparquet)){install.packages("nanoparquet")}

# Instalar librería para gráficos
if(!require(ggplot2)){install.packages("ggplot2")}
# Instalar librería de gráficos interactivos
if(!require(plotly)){install.packages("plotly")}; library(plotly) #lo llamamos directamente una vez instalado, en el caso que no lo hubiera estado

# definir carpetas de trabajo
datos_multidimensionales = 'https://github.com/rlagosb/taller_eiv/raw/refs/heads/main/datos_multidimensionales/'

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: rio

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘rio’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’, ‘writexl’, ‘R.utils’


Loading required package: arrow

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘arrow’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘assertthat’


Loading required package: nanoparquet

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return =

In [ ]:
# cargar datos

# concatenamos con paste0 la ubicación de github con el nombre de cada archivo *.parquet a trabajar
cubo <- rio::import(paste0(datos_multidimensionales, "Cubo_consultas_nuevas.xlsx"))

## 🏁 Discusión

1. ¿Qué métricas y dimensiones tiene esta tabla?
2. ¿Cuántas observaciones por año contiene?

In [ ]:
#echamos un vistazo a la base de datos cubo (clase y valores de las columnas, cantidad de columnas, cantidad de filas y primeras filas)
dplyr::glimpse(cubo)

## 🍎 Desafío

Construya una tabla de contingencia para reportar la oferta y demanda por año y especialidad



# Graficar series de tiempo
Construyamos un gráfico de la serie de tiempo con interactividad básica

In [ ]:
# Veamos que necesitamos procesar los datos para visualizarlos adecuadamente
# prompt: genera un gráfico de Oferta_consultas por periodo desglosado por Centro_siglas

# Generar el gráfico estático
ggplot_fig <- ggplot(cubo, aes(x = Periodo, y = Oferta_consultas, color = Centro_siglas)) +
  #en base a la estética (aes= eje x, y & el color de cada línea según las siglas del centro) definida, definimos que añadiremos una linea (geom_line)
  geom_line() +
  #definimos las etiquetas de los ejes y el títulod el gráfico
  labs(title = "Oferta de Consultas por Periodo y Centro",
       x = "Periodo",
       y = "Oferta de Consultas") +
  theme_minimal() #aplicamos una visualización con temática minimalista
#imprimimos el gráfico alojado
ggplot_fig

In [ ]:
#Así quedaría como en el ejemplo de python (ahora agregamos el argumento, "agrupar por"(group))
ggplot(cubo, aes(x = Periodo, y = Oferta_consultas, color = Centro_siglas, group = Centro_siglas)) +
  geom_line() +
  ggtitle("Oferta de Consultas por Periodo y Centro") +
  theme_minimal()

## 🏁 Discusión
¿Porqué se ve así la serie de tiempo *Oferta_consultas*?

In [ ]:
cubo%>%
  #agrupamos por periodo y centro
  dplyr::group_by(Periodo, Centro_siglas)%>%
  #resummimos para cada combinación de periodo y centro, una suma de todas las consultas en oferta (incluimos el argumento para ignorar/saltar NAs o valores inválidos, si hubiera)
  dplyr::summarise(Oferta_consultas=sum(Oferta_consultas, na.rm=T))%>%
    #en base a esta transformación, generamos el gráfico
ggplot(aes(x = Periodo, y = Oferta_consultas, color = Centro_siglas, group = Centro_siglas)) +
  geom_line() +
  #definimos las etiquetas de los ejes y el títulod el gráfico
  labs(title = "Oferta de Consultas por Periodo y Centro",
       x = "Periodo",
       y = "Oferta de Consultas") +
  theme_minimal()+
  # Añadimos etiquetas para cada punto
  geom_text(aes(label = Oferta_consultas), vjust = -0.5, size = 3)

## Gráfico interactivo

Para explorar las series, en vez de generar un script y un extracto para visualizar cada serie, podemos crear una función que reciba como parámetro la serie que queremos visualizar.

In [ ]:
# Generamos una función que toma como parámetro el nombre de la serie:
# Función para graficar series de datos
# y: Especialidad, o nombre de la columna a analizar como texto (por ejemplo: 'Consultas_producidas', 'Consultas_inasistencia', 'Oferta_consultas', 'Consultas_solicitadas', 'Lista_espera_inicial')
graficar_serie <- function(y) {

  # Agrupar los datos por Periodo y Centro_siglas
  cubo %>%
    dplyr::group_by(Periodo, Centro_siglas) %>%
    # Resumir los datos: sumar los valores de la columna especificada, ignorando NAs
    dplyr::summarise(
      suma_y = sum(.data[[y]], na.rm = TRUE),
      .groups = "drop" # Evita mensajes de agrupación posterior
    ) %>%
    # Generar el gráfico
    ggplot(aes(x = Periodo, y = suma_y, color = Centro_siglas, group = Centro_siglas)) +
    geom_line() + # Agregar líneas para cada Centro_siglas
    # Etiquetas de los ejes y título del gráfico
    labs(
      #concatenamos el título, reemplazando "_" por " " (espacio) para la variable de interés y que no se vea fea
      title = paste("Gráfico de", gsub("_"," ",y), "por Periodo y Centro"),
      x = "Periodo",
      y = y
    ) +
    theme_minimal() + # Aplicar un tema minimalista
    # Añadir etiquetas en cada punto con los valores de la serie
    geom_text(aes(label = suma_y), vjust = -0.5, size = 3)
}

In [ ]:
# Ejecutamos la función

graficar_serie('Consultas_producidas')

In [ ]:
# También podemos agregar un parámetro que permita filtrar por especialidad
graficar_serie_esp <- function(esp=NULL, y) {
  # Tenemos 2 argumentos o inputs en esta función= y (métrica) + esp (especialidad)
  # Agrupar los datos por Periodo y Centro_siglas
  cubo %>%
    # Filtrar por especialidad si se especifica
    {
      #se especifica esp en la función?
      if (!is.null(esp)) {
        #si es distinto a nulo, entonces se filtra por él
        dplyr::filter(., Especialidad == esp)
      } else {
        #de lo contrario, se sigue utilizando la base cubo como está
        .
      }
    } %>%
    dplyr::group_by(Periodo, Centro_siglas) %>%
    # Resumir los datos: sumar los valores de la columna especificada, ignorando NAs
    dplyr::summarise(
      suma_y = sum(.data[[y]], na.rm = TRUE),
       # Si la variable serie es igual a 'Lista_espera_inicial', divide los valores de esa columna entre 3.
       # si  # métrica no es aditiva al agregar por trimestre (3 meses)
      .groups = "drop" # Evita mensajes de agrupación posterior
    ) %>%
    # Generar el gráfico
    ggplot(aes(x = Periodo, y = suma_y, color = Centro_siglas, group = Centro_siglas)) +
    geom_line() + # Agregar líneas para cada Centro_siglas
    # Etiquetas de los ejes y título del gráfico
    labs(
      #concatenamos el título, reemplazando "_" por " " (espacio) para la variable de interés y que no se vea fea
      #Ahora agregamos especialidad
      title = paste(gsub("_"," ",y), "por Periodo y Centro (",esp,")"),
      x = "Periodo",
      y = y
    ) +
    theme_minimal() + # Aplicar un tema minimalista
    # Añadir etiquetas en cada punto con los valores de la serie
    geom_text(aes(label = suma_y), vjust = -0.5, size = 3)
}

In [ ]:
# Ejecutamos la función para un caso
# definimos el argumento esp, para que por descarte el otro quede como y
graficar_serie_esp('Oferta_consultas', esp='DERMATOLOGIA')

## 🍎 Desafíos

1. Explore otras series de tiempo cambiando los parámetros serie y/o especialidad.
2. Agregue un parámetro a la función *graficar_serie* que permita filtrar los datos para un año específico

## Desglosar por establecimiento

In [ ]:
# También podemos agregar un parámetro que permita filtrar por especialidad
graficar_serie_esp_facet <- function(esp=NULL, y, anio= NULL) {
  # Tenemos 2 argumentos o inputs en esta función= y (métrica) + esp (especialidad)
  # Agrupar los datos por Periodo y Centro_siglas
  cubo %>%
        dplyr::filter(Especialidad == esp) %>%
        dplyr::filter(Año == anio) %>%
    dplyr::group_by(Periodo, Centro_siglas) %>%
    # Resumir los datos: sumar los valores de la columna especificada, ignorando NAs
    dplyr::summarise(
      suma_y = sum(.data[[y]], na.rm = TRUE),
       # Si la variable serie es igual a 'Lista_espera_inicial', divide los valores de esa columna entre 3.
       # si  # métrica no es aditiva al agregar por trimestre (3 meses)
      .groups = "drop" # Evita mensajes de agrupación posterior
    ) %>%
    # Generar el gráfico
    ggplot(aes(x = Periodo, y = suma_y, group = 1, color= Centro_siglas)) +
    geom_line() +
    # Etiquetas de los ejes y título del gráfico
    labs(
      #concatenamos el título, reemplazando "_" por " " (espacio) para la variable de interés y que no se vea fea
      #Ahora agregamos especialidad
      title = paste(gsub("_"," ",y), "por Periodo y Centro (", esp, " & ", anio,")"),
      x = "Periodo",
      y = y
    ) +
    #_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_
    # trocear por centro
    facet_grid(~Centro_siglas)+
    #_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_
    # añadir paleta de colores por cada línea con una paleta de colores predeterminada
    scale_color_brewer(palette = "Set1") + # Cambiar colores para líneas y puntos según centros
    theme_minimal() + # Aplicar un tema minimalista
    # Añadir etiquetas en cada punto con los valores de la serie
    geom_text(aes(label = suma_y), vjust = -0.5, size = 3)+
    # Añadimos etiquetas por cada 100 unidades, partiendo desde 0 y terminando en 800 unidades, para que se parezca al gráfico de Python
    scale_y_continuous(breaks = seq(0, 800, by = 100))
}
graficar_serie_esp_facet(y='Consultas_solicitadas', esp='DERMATOLOGIA', anio=2021)

## 🏁 Discusión

1. ¿Qué ventajas y desventajas tiene generar las visualizaciones con un lenguaje de programación vs. realizarlos con Excel?
2. ¿Qué ventajas y desventajas tiene analizar los datos utilizando funciones como *graficar_series*?

# Graficar brechas

In [ ]:
graficar_brechas <- function(especialidad, anio) {
  # Filtrar los datos por especialidad y año
  df <- cubo %>%
    dplyr::filter(Especialidad == especialidad, Año == anio) %>%
    # Agrupar por Periodo
    dplyr::group_by(Periodo) %>%
    # Sumar las métricas
    dplyr::summarise(
      Oferta_consultas = sum(Oferta_consultas, na.rm = TRUE),
      Consultas_solicitadas = sum(Consultas_solicitadas, na.rm = TRUE),
      Lista_espera_inicial = sum(Lista_espera_inicial, na.rm = TRUE),
      .groups = "drop"
    ) %>%
    # Ajustar Lista_espera_inicial por trimestre
    dplyr::mutate(
      Lista_espera_inicial = Lista_espera_inicial / 3,
      Demanda_acumulada = cumsum(Consultas_solicitadas),
      Oferta_acumulada = cumsum(Oferta_consultas)
    )

  # Generar el gráfico
  ggplot(df) +
    #Group = 1: Para mantener el orden de los datos en las áreas (porque geom_area requiere continuidad en los valores de x).
    # Área para Lista_espera_inicial: área sombreada con azul claro, con transparencia al 50%
    geom_area(aes(x = Periodo, y = Lista_espera_inicial, group= 1), alpha = 0.5, fill ="lightblue") +
    # Área para Demanda_acumulada: área sombreada con verde claro, con transparencia al 50%
    geom_area(aes(x = Periodo, y = Demanda_acumulada+Lista_espera_inicial, group= 1), alpha = 0.5, fill ="lightgreen") +
    # Línea para Oferta_acumulada: línea coloreada con azul oscuro y grosor de 1
    geom_line(aes(x = Periodo, y = Oferta_acumulada, group= 1), linewidth = 1, color = "darkblue") +
    # Punto para Oferta_acumulada: punto coloreado con azul oscuro y grosor de 2
    geom_point(aes(x = Periodo, y = Oferta_acumulada, group= 1), color="darkblue", size = 2) +
    # Etiquetas del gráfico
    labs(
      title = paste("Brechas por Periodo (", especialidad, " al ",anio, ")", sep = ""),
      x = "Periodo",
      y = "Cantidad",
      fill = "Métricas",
      color = "Métricas",
      #Añadimos una nota explicativa a falta de una leyenda, e introducimos un salto de línea (\n) para  poder visualizarla
      caption= "Área azul claro= Lista espera inicial; Área verde claro= Demanda acumulada;\nOferta acumulada= Punto azul oscuro"
    ) +
    theme_minimal()
}


In [ ]:
unique(cubo$Especialidad)

In [ ]:
graficar_brechas('PSIQUIATRIA ADULTO',2021)

Finalmente generamos una función para desglosar las brechas por establecimiento

In [ ]:
graficar_brechas_centro <- function(especialidad, anio) {
  # Filtrar los datos por especialidad y año
  cubo %>%
    dplyr::filter(Especialidad == especialidad, Año == anio) %>%
    #_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_
    # Agrupar por Periodo y Centro
    dplyr::group_by(Periodo, Centro_siglas) %>%
    #_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_
    # Sumar las métricas
    dplyr::summarise(
      Oferta_consultas = sum(Oferta_consultas, na.rm = TRUE),
      Consultas_solicitadas = sum(Consultas_solicitadas, na.rm = TRUE),
      Lista_espera_inicial = sum(Lista_espera_inicial, na.rm = TRUE),
      .groups = "drop"
    ) %>%
    #_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_
    # No utilizamos agrupación por unidades ni periodos para la lista de espera inicial, por su definición
    dplyr::mutate(
    # Ajustar Lista_espera_inicial por trimestre
      Lista_espera_inicial = Lista_espera_inicial / 3)%>%
    #_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_
    dplyr::group_by(Centro_siglas)%>%
    # En cambio, sí generamos las demandas y ofertas acumuladas por centro. De lo contrario las tendríamos infladas porque representarían al conjunto de centros de la base
    dplyr::mutate(
      # Va acumulando por trimestre
      Demanda_acumulada = cumsum(Consultas_solicitadas),
      Oferta_acumulada = cumsum(Oferta_consultas)
    )%>%
    dplyr::ungroup()%>%
    # Reemplazamos el guíon de periodo por un salto de línea, para mayor claridad en el gráfico
    dplyr::mutate(Periodo=gsub("-", "\n", Periodo))%>%
  # Generar el gráfico
  ggplot() +
    #Group = 1: Para mantener el orden de los datos en las áreas (porque geom_area requiere continuidad en los valores de x).
    # Área para Demanda_acumulada: área sombreada con rojo claro (#FF7F7F), con transparencia al 50%. Para ello sumamos la lista de espera inicial a la demanda, para apilarla
    geom_area(aes(x = Periodo, y = Demanda_acumulada+Lista_espera_inicial, group= 1), alpha = 0.3, fill ="#FF7F7F") +
    # Área para Lista_espera_inicial: área sombreada con azul claro, con transparencia al 50%
    geom_area(aes(x = Periodo, y = Lista_espera_inicial, group= 1), alpha = 0.5, fill ="lightblue") +
    # Línea para Oferta_acumulada: línea coloreada con azul oscuro y grosor de 1
    geom_line(aes(x = Periodo, y = Oferta_acumulada, group= 1), linewidth = 1, color = "darkblue") +
    # Punto para Oferta_acumulada: punto coloreado con azul oscuro y grosor de 2
    geom_point(aes(x = Periodo, y = Oferta_acumulada, group= 1), color="darkblue", size = 2) +
    #_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_
    # Segmentamos los gráficos por centro
    facet_grid(~Centro_siglas)+
    #_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_
    # Añadimos etiquetas por cada 200 unidades, partiendo desde 0 y terminando en 1600 unidades, para que se parezca al gráfico de Python
    scale_y_continuous(breaks = seq(0, 1600, by = 200)) +
    #_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_
    # Etiquetas del gráfico
    labs(
      title = paste("Brechas por Periodo (", especialidad, " al ",anio, ")", sep = ""),
      x = "Periodo",
      y = "Cantidad",
      fill = "Métricas",
      color = "Métricas",
      #Añadimos una nota explicativa a falta de una leyenda, e introducimos un salto de línea (\n) para  poder visualizarla
      caption= "Área azul claro= Lista espera inicial; Área rojo claro= Demanda acumulada;\nOferta acumulada= Punto azul oscuro; Etiqueta azul oscura= Recuento oferta acumulada"
    ) +
    theme_minimal()+
    # Añadimos etiquetas para cada punto de la oferta
    geom_text(aes(x = Periodo, y = Oferta_acumulada, group= 1,label = Oferta_acumulada), color="darkblue", vjust = -0.5, size = 3)
}

graficar_brechas_centro('PSIQUIATRIA ADULTO',2021)

# Extra:
En R, en general trabajamos con bases de datos en formato largo, en que en una columna se agrupan los valores, y cada fila es la combinación de la especialidad, centro, año y tipo de métrica.

In [ ]:
graficar_brechas_centro_long <- function(especialidad, anio) {
  # Filtrar los datos por especialidad y año
cubo %>%
    dplyr::filter(Especialidad == especialidad, Año == anio) %>%
    dplyr::group_by(Periodo, Centro_siglas) %>%
    dplyr::summarise(
      Oferta_consultas = sum(Oferta_consultas, na.rm = TRUE),
      Consultas_solicitadas = sum(Consultas_solicitadas, na.rm = TRUE),
      Lista_espera_inicial = sum(Lista_espera_inicial, na.rm = TRUE),
      .groups = "drop"
    ) %>%
    dplyr::mutate(
      Lista_espera_inicial = Lista_espera_inicial / 3
    ) %>%
    dplyr::group_by(Centro_siglas) %>%
    dplyr::mutate(
      Demanda_acumulada = cumsum(Consultas_solicitadas),
      Oferta_acumulada = cumsum(Oferta_consultas)
    ) %>%
    dplyr::ungroup() %>%
    #_#_#_#_#_#_#_#_#_#_#_#_
    # Reformateamos la base de datos en formato largo.
    # Generamos la columna métrica. En ella,
    tidyr::pivot_longer(
      cols = c(Lista_espera_inicial, Demanda_acumulada, Oferta_acumulada),
      names_to = "metrica",
      values_to = "value"
    ) %>%
    # Reemplazamos "_" por " " en la métrica
    dplyr::mutate(metrica= gsub("_","\n",metrica))%>%
    #_#_#_#_#_#_#_#_#_#_#_#_
    # Reemplazamos el guíon de periodo por un salto de línea, para mayor claridad en el gráfico
    dplyr::mutate(Periodo=gsub("-", "\n", Periodo), levels = gsub("-", "\n", Periodo))%>%
    #_#_#_#_#_#_#_#_#_#_#_#_
  # Generar el gráfico
  ggplot() +
    geom_area(
      data = . %>% dplyr::filter(metrica %in% c("Lista\nespera\ninicial", "Demanda\nacumulada")),
      aes(x = Periodo, y = value, fill = metrica, group = metrica),
      alpha = 0.4,
      position = "stack"
    ) +
    geom_line(
      data = . %>% dplyr::filter(metrica == "Oferta\nacumulada"),
      aes(x = Periodo, y = value, group = Centro_siglas),
      linewidth = 1, color = "darkblue"
    ) +
    geom_point(
      data = . %>% dplyr::filter(metrica == "Oferta\nacumulada"),
      aes(x = Periodo, y = value, group = Centro_siglas),
      color = "darkblue", size = 2
    ) +
    geom_text(
      data = . %>% dplyr::filter(metrica == "Oferta\nacumulada"),
      aes(x = Periodo, y = value, label = value),
      color = "darkblue", vjust = -0.5, size = 3
    ) +
    facet_grid(~Centro_siglas) +
    scale_y_continuous(breaks = seq(0, 1600, by = 200)) +
    labs(
      title = paste("Brechas por Periodo (", especialidad, " al ", anio, ")", sep = ""),
      x = "Periodo",
      y = "Cantidad",
      fill = "Métricas",
      caption = "Área azul claro = Lista espera inicial; Área rojo claro = Demanda acumulada;\nOferta acumulada = Línea y puntos azul oscuro"
    ) +
    scale_fill_manual(
      values = c(
        "Lista\nespera\ninicial" = "lightblue",
        "Demanda\nacumulada" = "#FF7F7F"
      )
    ) +
    theme_minimal()
}


In [ ]:
# Construimos el gráfico para una especialidad

graficar_brechas_centro_long('PSIQUIATRIA ADULTO', 2021)

## 🍎 Desafíos

1. Ejecute *generar_brechas_centros* con otra especialidad

2. Modifique alguna de las visualizaciones incorporando otras variables disponibles en el cubo.
2. Genere otra visualización de las brechas que le parezca interesante para contrastar la demanda y oferta.

## 🏁 Discusión

1. ¿Qué tendría que hacer para reproducir el análisis con datos de 2024?
4. ¿Qué tendría que hacer para replicar el análisis en otro Servicio de Salud?
4. ¿Qué tendría que hacer si necesitara incorporar variables que no están en el cubo (ejemplo: solicitudes de interconsultas GES)?

# Plotly❓

Por desgracia, la integración de R en Colab todavía no está completa. Por esta razón no se despliegan gráficos interactivos ni tablas o módulos que contengan HTML por razones de seguridad 😞.

De todas formas, dejamos un enlace de esta potente librería para su conocimiento: [**GGplotly**](https://www.datacamp.com/datalab/w/566df73a-45bc-488a-8ccf-0e7ade9ddb10/edit/4_Ejercicio_visualizaciones_y_reportes(R).ipynb)